In [29]:
from functools import wraps

class Select(object):
    _sql = "select {field} from {table} where {filter} {group}"

    def __init__(self, fld, flt, grp):
        self._field = fld
        self._filter = flt
        self._group = grp

    def __call__(self, fn):
        @wraps(fn)
        def wrapped(*args, **kwargs):
            fieldStr = ','.join(self._field)
            filterStr = ','.join(self._filter)
            groupStr = ','.join(self._group)
            groupStr = "" if len(groupStr) <= 0 else "group by " + groupStr
            return self._sql.format(field=fieldStr,
                                    table=fn(*args, **kwargs),
                                    filter=filterStr,
                                    group=groupStr)

        return wrapped


class SubQuery(object):
    _sql = "select {field} from ({subquery}) {alias}"

    def __init__(self, field, alias):
        self._field = field
        self._alias = alias

    def __call__(self, fn):
        @wraps(fn)
        def wrapped(*args, **kwargs):
            f = [self._alias + '.' + f for f in self._field]
            return self._sql.format(field=','.join(f), 
                                    subquery=fn(*args, **kwargs), 
                                    alias=self._alias)

        return wrapped


class Join(object):
    _sql = """
    select {field} from ({t1}) t1
    join ({t2}) t2 on({on})
    """

    def __init__(self, field, t1, t2, on):
        self._field = field
        self._t1 = t1
        self._t2 = t2
        self._on = on

    def __call__(self, fn):
        @wraps(fn)
        def wrapped(*args, **kwargs):
            return self._sql.format(field=','.join(self._field), 
                                    t1=self._t1, 
                                    t2=self._t2, 
                                    on=' '.join(self._on))

        return wrapped


class LeftJoin(object):
    _sql = """
    select {field} from ({t1}) t1
    left join ({t2}) t2 on({on})
    """

    def __init__(self, field, t1, t2, on):
        self._field = field
        self._t1 = t1
        self._t2 = t2
        self._on = on

    def __call__(self, fn):
        @wraps(fn)
        def wrapped(*args, **kwargs):
            return self._sql.format(field=','.join(self._field), 
                                    t1=self._t1, 
                                    t2=self._t2, 
                                    on=' '.join(self._on))

        return wrapped


class RightJoin(object):
    _sql = """
    select {field} from ({t1}) t1
    right join ({t2}) t2 on({on})
    """

    def __init__(self, field, t1, t2, on):
        self._field = field
        self._t1 = t1
        self._t2 = t2
        self._on = on

    def __call__(self, fn):
        @wraps(fn)
        def wrapped(*args, **kwargs):
            return self._sql.format(field=','.join(self._field), 
                                    t1=self._t1, 
                                    t2=self._t2, 
                                    on=' '.join(self._on))

        return wrapped

In [30]:
@SubQuery(field=['1111'],alias='aaaa')
@Select(fld=['*'], flt=["1=1"], grp=[])
def table():
    return 'table'

table()

'select aaaa.1111 from (select * from table where 1=1 ) aaaa'

In [31]:
@Select(fld=['*'], flt=["1=1"], grp=[])
def table1():
    return 'table1'

@Select(fld=['*'], flt=["1=1"], grp=[])
def table2():
    return 'table2'

s2=table1()
s3=table2()

@Join(field=['*'],t1=s2,t2=s3,on=['t1.id=t2.id'])
def join():
    return ''

join()

'\n    select * from (select * from table1 where 1=1 ) t1\n    join (select * from table2 where 1=1 ) t2 on(t1.id=t2.id)\n    '